This notebook shows how you can test your model for inference. You can use this notebook as a starting point for your inference activities.

In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow import MlflowClient
import pandas as pd

In [2]:
load_dotenv()
{% if cookiecutter.use_minio == 'yes' %}
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("J_MLFLOW_S3_ENDPOINT_URL")
{% endif %}

True

In [4]:
def get_model_for_inference(model_name: str, version: str) -> (
        mlflow.pyfunc.PyFuncModel):
    """
    This function gets the model from the MLFlow registry based on the model_name
    and version provided.
    """
    model_version = client.get_model_version(model_name, version)
    artifact_uri = model_version.source
    {% if cookiecutter.use_minio == 'yes' %}
    print(f"Artifact URI: {artifact_uri}")
    loaded_model = mlflow.pyfunc.load_model(artifact_uri)
    {% else %}
    final_artifact_path = artifact_uri.replace("/mlflow_artifacts", "./mlflow_artifacts", 1)
    print(f"Artifact URI: {final_artifact_path}")
    loaded_model = mlflow.pyfunc.load_model("../../" + final_artifact_path)
    {% endif %}
    print("model loaded")
    return loaded_model

# This is important as we need to tell this local MLFlow that there is already a MLFlow instance running 
# in docker exposed at this URI: J_MLFLOW_SERVER_URI
mlflow.set_tracking_uri(os.getenv('J_MLFLOW_SERVER_URI'))
client = MlflowClient(mlflow.get_tracking_uri())

# Provide your model name and the version you would like to test
model_name = "tiny_iris_classifier"
version = "1"

loaded_model = get_model_for_inference(model_name, version)

# Prepare your data for inference
input_data = sample_data = pd.DataFrame([[5.1, 3.5, 1.4, 0.2]],
      columns=["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"])

# Perform inference
prediction = loaded_model.predict(pd.DataFrame(input_data))
print("Prediction:", prediction)

Artifact URI: /mlflow_artifacts/0/998e099fd5f2480eb5466396e6419f25/artifacts/iris_model


2025/01/31 11:34:37 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.0, required: mlflow==2.20.1)
 - numpy (current: 2.2.2, required: numpy==1.26.4)
 - pandas (current: 2.2.2, required: pandas==2.1.4)
 - psutil (current: 6.1.1, required: psutil==6.1.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


model loaded
Prediction: [0]


/home/yogesh/miniforge3/envs/mlops/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
